In [1]:
import itertools
import os
import time
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import linalg as la
import networkx as nx
from scipy.linalg import logm, expm
from scipy.sparse.linalg import eigsh, svds
from pandas.plotting._matplotlib.style import get_standard_colors
colors = get_standard_colors(num_colors=13)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import ray
ray.init()

2024-01-16 15:10:30,320	INFO worker.py:1724 -- Started a local Ray instance.


Python version:,3.11.5
Ray version:,2.9.0


## Fréchet Online Changepoint Detection Algorithm

In [2]:
def nearestPD(A):
    """Find the nearest positive-definite matrix to input
    https://stackoverflow.com/a/43244194/13097140
    A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
    credits [2].

    [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd

    [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
    matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
    """

    B = (A + A.T) / 2
    _, s, V = la.svd(B)

    H = np.dot(V.T, np.dot(np.diag(s), V))

    A2 = (B + H) / 2

    A3 = (A2 + A2.T) / 2

    if isPD(A3):
        return A3

    spacing = np.spacing(la.norm(A))
    I = np.eye(A.shape[0])
    k = 1
    while not isPD(A3):
        mineig = np.min(np.real(la.eigvals(A3)))
        A3 += I * (-mineig * k**2 + spacing)
        k += 1

    return A3


def isPD(B):
    """Returns true when input is positive-definite, via Cholesky"""
    try:
        _ = la.cholesky(B)
        return True
    except la.LinAlgError:
        return False

In [3]:
class ChangePointDetector:
    def __init__(self, G_list, T0=100, trimming=0.3, alpha=0.05, gamma=0.35):
        """Detect multiple change points in a sequence of graph snapshots
        Parameters
        ----------
        G_list : `list`, len=(n_snapshots)
            List of networkx graphs.
        T0: `int`, default=100
            The first possible change point location
        epsilon: `float`, default=0.01
            The positive eigenvalue used to replace the zero eigenvalues of a Laplacian.
        alpha: `float`, default=0.05
            Significance level
        Returns
        -------
        """        
        self.logG_seq = logG_sequence(G_list)
#         print(f"Finished computing the nearest SPDs of the input sequence of {len(G_list)} graphs")
        self.T0 = T0
        self.trimming = trimming
        self.alpha = alpha
        self.gamma = gamma
        self.c = [2.5050 if alpha==0.05 else 2.2433 if alpha==0.1 else 3.0475 if alpha==0.01 else np.nan][0]
        self.cp_detected = []
        self.test_stats = []
        self.result = None
    
    def frechet_stats_test(self, logG_seq):
        T = logG_seq.shape[0]
        T0 = self.T0
        trimming = int(self.trimming*T0)
#         print(f"Testing a sequence of {T} graphs; initial phase spans {T0} time steps")
        test_stats_seq, sigma2 = test_statistics_online_two_pass(logG_seq, T0, trimming)
        boundary_seq = np.array([boundary_function(T0, k, self.gamma) for k in range(1, T-T0+1)])
        ratio_seq = np.abs(test_stats_seq / boundary_seq / np.sqrt(sigma2))[trimming:]
        change_points = np.where(ratio_seq > self.c)[0] + trimming
        if len(change_points) == 0:
            return False, -1, test_stats_seq, boundary_seq, ratio_seq, sigma2
        else:
            return True, change_points[0]+T0, test_stats_seq, boundary_seq, ratio_seq, sigma2

    def change_point_detection(self, logG_seq=None):
        if logG_seq is None:
            logG_seq = self.logG_seq
        exist, loc, test_stats_seq, boundary_seq, ratio_seq, sigma2 = self.frechet_stats_test(logG_seq)
        self.result = exist, loc, test_stats_seq, boundary_seq, ratio_seq, sigma2
#         print(f"Finishing change point detection: {exist}")
        
    def restart(self):
        self.cp_detected = []
        self.test_stats = []

In [4]:
def dist(A, order=2):
#     return ((A ** 2).sum(axis=(1, 2)) ** (order//2)).sum()
    if order == 2:
        return np.einsum('ijk,ijk->...', A, A)
    else:
        return (np.einsum('ijk,ijk->i', A, A)**(order//2)).sum()

@ray.remote
def compute_logG(G, max_nodes, directed=False, epsilon=1e-2):
    if G.number_of_nodes() < max_nodes:
        G.add_nodes_from([-i for i in range(G.number_of_nodes(), max_nodes)])
    if directed:
        L = nx.directed_laplacian_matrix(G)
    else:
        L = nx.laplacian_matrix(G).toarray()
    L = nearestPD(L)
#     vals, vecs = eigh(L)
#     logG_t = vecs @ np.diag(np.log(np.where(vals > 0, vals, 1e-2))) @ vecs.T
    try:
        vecs_u, vals, vecs_v = svds(L, int(max_nodes*0.8), which="LM")
        logG_t = vecs_u @ np.diag(np.log(vals)) @ vecs_v
    except:
        logG_t = logm(L)
    return logG_t

def logG_sequence(G_list, epsilon=1e-2):
    directed = G_list[0].is_directed()
    max_nodes = max([G.number_of_nodes() for G in G_list])
    sol_logG_seq = [compute_logG.remote(G, max_nodes, directed, epsilon) for G in G_list]
    logG_seq = ray.get(sol_logG_seq)
    return np.stack(logG_seq, axis=0)

def test_statistics_online_two_pass(X_seq, T0, trimming):
    if isinstance(X_seq, list):
        T = len(X_seq)
    else:
        T = X_seq.shape[0]
        
    test_stats_seq = np.zeros(T-T0, dtype=np.float64)
    mu01 = X_seq[:T0].mean(axis=0)
    V01 = dist(X_seq[:T0] - mu01) / T0 # compute the second part V[0, 1]
    sigma2 = dist(X_seq[:T0] - mu01, order=4) / T0 - V01**2
    
    for idx, tau in enumerate(range(T0, T)):
        mu1u = X_seq[T0:tau+1].mean(axis=0)
        V1u = dist(X_seq[T0:tau+1] - mu1u) / (tau+1-T0) # compute the first part V[1, u]
        VC1u = dist(X_seq[T0:tau+1] - mu01) / (tau+1-T0)
        VC01 = dist(X_seq[:T0] - mu1u) / T0
        contaminate = VC1u - V1u + VC01 - V01
        test_stats_seq[idx] = (tau+1-T0) * (V1u - V01) # + (VC1u - V1u + VC01 - V01)# add the contaminated term
        
    return test_stats_seq, sigma2

def boundary_function(n, k, gamma=0.35): # boundary function for comparing with the test statistic
    return np.sqrt(n)*(1+k/n)*(k/(n+k))**gamma

## functions to generate dynamic SBM based on given changepoints 

In [5]:
def create_SBM_parameters(n=50, p11=0.25, p12=0.05, change_type=1, minimum_average_node_per_block=5):
    if change_type == 1:
        num_blocks = 2 ** np.arange(np.log(n // minimum_average_node_per_block) / np.log(2))
        num_blocks = num_blocks.astype(int)
        size_prob_parameters = []
        for n_block in num_blocks:
            sizes = [n // n_block] * (n_block - 1)
            sizes.append(n - sum(sizes))
            p = (p11 - p12) * np.eye(n_block) + p12 * np.ones((n_block, n_block))
            size_prob_parameters.append([sizes, p])
    else:
        p11 = p11 * 50 / n
        p12 = p12 * (50 / n) ** (1/2)
        n_block = 4
        size_prob_parameters = []
        sizes = [n // n_block] * (n_block - 1)
        sizes.append(n - sum(sizes))
        for q11 in np.linspace(p11*0.5, p11*2, 3):
            for q12 in np.linspace(p12*0.5, p12*2, 3):
                p = (q11 - q12) * np.eye(n_block) + q12 * np.ones((n_block, n_block))
                size_prob_parameters.append([sizes, p])
                
    return size_prob_parameters

In [6]:
def generate_snapshot_on_consecutive_networks(G_old, G_new, alpha=0):
    if alpha == 0:
        return G_old
    if alpha == 1:
        return G_new
    num_nodes = G_old.number_of_nodes()
    G_interp = nx.from_edgelist([edge for u, edge in zip(np.random.rand(G_old.number_of_edges()), nx.to_edgelist(G_old)) if u < alpha] + \
                            [edge for u, edge in zip(np.random.rand(G_new.number_of_edges()), nx.to_edgelist(G_new)) if u > alpha])
    connected = nx.is_connected(G_interp)
    while G_interp.number_of_nodes() < num_nodes or connected == False:
        G_interp = nx.from_edgelist([edge for u, edge in zip(np.random.rand(G_old.number_of_edges()), nx.to_edgelist(G_old)) if u < alpha] + \
                            [edge for u, edge in zip(np.random.rand(G_new.number_of_edges()), nx.to_edgelist(G_new)) if u > alpha])
        connected = nx.is_connected(G_interp)
    return G_interp

In [7]:
def generate_dynamic_SBM(T=100, cps=[], n=50, p11=0.25, p12=0.05, alpha=0.001, directed=False, change_type=1):
    cp_counter = 0
    G_prev = None
    G_curr = None
    G_list = []
    size_prob_parameters = create_SBM_parameters(n, p11, p12, change_type=change_type)
    size_prob_index_prev = None
    size_prob_index_curr = np.random.choice(len(size_prob_parameters))
    for t in range(T):
        cp = False
        connected = False
        if cp_counter < len(cps) and t == cps[cp_counter]:
            cp = True
            if size_prob_index_prev is None:
                size_prob_index_prev = size_prob_index_curr
            while size_prob_index_curr == size_prob_index_prev:
                size_prob_index_curr = np.random.choice(len(size_prob_parameters))
            cp_counter += 1
        sizes, p = size_prob_parameters[size_prob_index_curr]
        while connected == False:
            G_curr = nx.stochastic_block_model(sizes, p, directed=directed)
            connected = nx.is_connected(G_curr) # this is to ensure that each graph snapshot is connected
        if G_prev is not None and cp == False:
            G_curr = generate_snapshot_on_consecutive_networks(G_prev, G_curr, alpha)
        G_list.append(G_curr)
        G_prev = G_curr
    
    return G_list, cps

In [8]:
def simulation(num_runs=100, change=False, alpha=0.05, gamma=0.3, T=100, change_type=1):
    T0 = T//2 # the earliest time of a possible change point
    if change == False:
        cps = []
    else:
        cps = [np.random.randint(2*T0//3+T//3, T0//3+2*T//3)]
        
    n = 50
    directed = False
    G_list, cps = generate_dynamic_SBM(T, cps, n, directed=directed, change_type=change_type)
    Synthetic_cp = ChangePointDetector(G_list, T0=T0, alpha=alpha, gamma=gamma)
    Synthetic_cp.change_point_detection()
    exist, loc, test_stats_seq, boundary_seq, ratio_seq, sigma2 = Synthetic_cp.result
    
    return exist, loc, cps[0] if len(cps)==1 else float('inf')

In [9]:
ALPHA = [0.01, 0.05, 0.1]
GAMMA = [0.25, 0.35, 0.45]
TIME = [100, 200]
num_runs = 1000

sizes = {}
powers = {}
accuracys = {}
delays = {}

for alpha in ALPHA:
    for gamma in GAMMA:
        for T in TIME:
            print(f"Experiment with alpha={alpha}, gamma={gamma}, T={T}")
            change = False
            exist_array = np.zeros(num_runs)
            loc_detect_array = np.zeros(num_runs)
            loc_true_array = np.zeros(num_runs)
            for run_id in range(num_runs):
                exist, loc_detect, loc_true = simulation(num_runs, change=change, alpha=alpha, gamma=gamma, T=T, change_type=1)
                exist_array[run_id] = exist
                loc_detect_array[run_id] = loc_detect
                loc_true_array[run_id] = loc_true    
            print(f"size: {exist_array.mean()}")
            sizes[(alpha, gamma, T)] = exist_array.mean(), exist_array.std()
            
            change = True
            exist_array = np.zeros(num_runs)
            loc_detect_array = np.zeros(num_runs)
            loc_true_array = np.zeros(num_runs)
            for run_id in range(num_runs):
                exist, loc_detect, loc_true = simulation(num_runs, change=change, alpha=alpha, gamma=gamma, T=T)
                exist_array[run_id] = exist
                loc_detect_array[run_id] = loc_detect
                loc_true_array[run_id] = loc_true
                
            print(f"power: {exist_array.mean()}, delay: {np.where(loc_detect_array >= loc_true_array, loc_detect_array - loc_true_array, 0)[loc_detect_array >= loc_true_array].mean()}")            
            powers[(alpha, gamma, T)] = exist_array.mean(), exist_array.std()
            accuracys[(alpha, gamma, T)] = (loc_detect_array[exist_array==True]==loc_true_array[exist_array==True]).mean(), (loc_detect_array[exist_array==True]==loc_true_array[exist_array==True]).std()
            delays[(alpha, gamma, T)] = np.where(loc_detect_array >= loc_true_array, loc_detect_array - loc_true_array, 0)[loc_detect_array >= loc_true_array].mean(), np.where(loc_detect_array >= loc_true_array, loc_detect_array - loc_true_array, 0)[loc_detect_array >= loc_true_array].std()

Experiment with alpha=0.01, gamma=0.25, T=100
size: 0.083
power: 0.863, delay: 6.041509433962264
Experiment with alpha=0.01, gamma=0.25, T=200
size: 0.014
power: 0.893, delay: 11.249142857142857
Experiment with alpha=0.01, gamma=0.35, T=100
size: 0.173
power: 0.862, delay: 5.856330014224751
Experiment with alpha=0.01, gamma=0.35, T=200


KeyboardInterrupt: 